In [1]:
using Plots

In [6]:
using MAT

┌ Info: Precompiling MAT [23992714-dd62-5051-b70f-ba57cb901cac]
└ @ Base loading.jl:1278


In [82]:
eps = matread("test_iplp.mat")["eps"]
A = matread("test_iplp.mat")["A"]
c = reshape(matread("test_iplp.mat")["c"], :)
b = reshape(matread("test_iplp.mat")["b"], :)
eta = matread("test_iplp.mat")["eta"]
kmax = Int(matread("test_iplp.mat")["kmax"])

m, n = size(A)
@assert size(c) == (n,)
@assert size(b) == (m,)
m, n

(5, 7)

In [239]:
module Optimizers
using LinearAlgebra
function ipm_lm(
        A::Matrix{Float64},
        b::Vector{Float64},
        c::Vector{Float64},
        eta::Float64,
        eps::Float64,
        kmax::Int64,
        x0::Vector{Float64},
        lambda0::Vector{Float64},
        s0::Vector{Float64}
    )::Tuple{
        Array{Float64,1},Array{Float64,1},Array{Float64,1},Float64,Float64,Int64
    }
    m, n = size(A)
    @assert size(c, 1) == n
    @assert size(b, 1) == m
    e = ones(n)
    
    k = 0
    
    xk::Vector{Float64} = (x0)
    lambdak::Vector{Float64} = (lambda0)
    sk ::Vector{Float64}= (s0)
    etak = (eta)
    
    mu0 = ((xk' * sk) / n)
    
    while true
        if k > kmax
            break
        end
        
        muk = ((xk' * sk) / n)
        
        if muk <= eps*mu0
            break
        end
        
        F(x::Vector{Float64}, lambda::Vector{Float64}, s::Vector{Float64}) = [
            A' * lambda + s - c
            A * x - b
            diagm(x) * diagm(s) * e
        ]
        JF(x::Vector{Float64}, lambda::Vector{Float64}, s::Vector{Float64}) = [
            zeros(n, n)              A'           I(n)
                      A     zeros(m, m)    zeros(m, n)
                diagm(s)    zeros(n, m)       diagm(x)
        ]

        # 1. (Prediction)
        affine_scaling_step_k = (JF(xk, lambdak, sk) \ -F(xk, lambdak, sk))
        delta_xk_aff = affine_scaling_step_k[1:n]
        delta_lambdak_aff = affine_scaling_step_k[n+1:n+m]
        delta_sk_aff = affine_scaling_step_k[n+m+1:n+m+n]

        # 2.
        alpha_P_aff = (min(1.0, min([(-(xk[i])/delta_xk_aff[i]) for i in 1:n if delta_xk_aff[i] < 0.0]...)))
        alpha_D_aff = (min(1.0, min([(-(sk[i])/delta_sk_aff[i]) for i in 1:n if delta_sk_aff[i] < 0.0]...)))
        
        # 3.
        muk_aff = (((xk + alpha_P_aff * delta_xk_aff)' * (sk + alpha_D_aff * delta_sk_aff)) / n)
        sigmak = ((muk_aff / muk)^3)

        # 4. (Correction)
        corrector_step_k = (JF(xk, lambdak, sk) \ (-F(xk, lambdak, sk) + [
                zeros(n)
                zeros(m)
                -diagm(delta_xk_aff)*diagm(delta_sk_aff)*e + sigmak * muk * e
        ]))
        delta_xk = corrector_step_k[1:n]
        delta_lambdak = corrector_step_k[n+1:n+m]
        delta_sk = corrector_step_k[n+m+1:n+m+n]

        # 5.
        alpha_P_hat = (min(1, min([(-(xk[i])/delta_xk[i]) for i in 1:n if delta_xk[i] < 0.0]...)))
        alpha_D_hat = (min(1, min([(-(sk[i])/delta_sk[i]) for i in 1:n if delta_sk[i] < 0.0]...)))

        alpha_P = min(1.0, etak * alpha_P_hat)
        alpha_D = min(1.0, etak * alpha_D_hat)

        # 6.
        xk = (xk) + (alpha_P * delta_xk)
        lambdak = (lambdak) + (alpha_D * delta_lambdak)
        sk = (sk) + (alpha_D * delta_sk)
        
        k += 1
    end
    
    muk = ((xk' * sk) / n)
    fk = (c' * xk)
    
    return xk, lambdak, sk, fk, muk, k
end
end
using Main.Optimizers

x0 = ones(n)
lambda0 = ones(m)
s0 = ones(n)

@time xk, lambdak, sk, fk, muk, k = Optimizers.ipm_lm(A, b, c, eta, eps, kmax, x0, lambda0, s0)

@show xk
@show lambdak
@show sk
@show fk
@show muk
@show k

nothing

  0.001085 seconds (1.21 k allocations: 222.906 KiB)
xk = [4.012053859498594e-7, 3.1518715973353527e-7, 9.999998963709144, 15.99999887297977, 3.000000084447843, 4.0120538594986446e-7, 3.151871597335344e-7]
lambdak = [7.607195153767026e-8, -1.1590980448188076e-8, 8.132554521509954e-10, -0.49999995419098814, -0.4999999490875558]
sk = [0.5000000332735677, 0.500000028170139, 3.010630336158069e-9, 9.06735623220153e-8, 7.826932642167679e-8, 0.50000003327357, 0.5000000281701377]
fk = 7.163925456833946e-7
muk = 3.474405387454147e-7
k = 7


In [240]:
@code_warntype Optimizers.ipm_lm(A, b, c, eta, eps, kmax, x0, lambda0, s0)

Variables
  #self#::Core.Compiler.Const(Main.Optimizers.ipm_lm, false)
  A::Array{Float64,2}
  b::Array{Float64,1}
  c::Array{Float64,1}
  eta::Float64
  eps::Float64
  kmax::Int64
  x0::Array{Float64,1}
  lambda0::Array{Float64,1}
  s0::Array{Float64,1}
  @_11::Int64
  m::Int64
  n::Int64
  e::Array{Float64,1}
  k::Int64
  xk@_16::Core.Box
  lambdak::Array{Float64,1}
  sk@_18::Core.Box
  etak::Float64
  mu0::Float64
  muk::Float64
  fk::Float64
  #1::Main.Optimizers.var"#1#11"{Array{Float64,1}}
  #2::Main.Optimizers.var"#2#12"{Array{Float64,1}}
  #3::Main.Optimizers.var"#3#13"{Array{Float64,1}}
  #4::Main.Optimizers.var"#4#14"{Array{Float64,1}}
  #5::Main.Optimizers.var"#5#15"{Array{Float64,1}}
  #6::Main.Optimizers.var"#6#16"{Array{Float64,1}}
  #7::Main.Optimizers.var"#7#17"{Array{Float64,1}}
  #8::Main.Optimizers.var"#8#18"{Array{Float64,1}}
  F::Main.Optimizers.var"#F#9"{Array{Float64,2},Array{Float64,1},Array{Float64,1},Array{Float64,1}}
  JF::Main.Optimizers.var"#JF#10"{Array{Fl

In [235]:
@code_lowered Optimizers.ipm_lm(A, b, c, eta, eps, kmax, x0, lambda0, s0)

CodeInfo(
1 ──        Core.NewvarNode(:(e))
│           Core.NewvarNode(:(k))
│           xk@_16 = Core.Box()
│           Core.NewvarNode(:(lambdak))
│           sk@_18 = Core.Box()
│           Core.NewvarNode(:(etak))
│           Core.NewvarNode(:(mu0))
│           Core.NewvarNode(:(muk))
│           Core.NewvarNode(:(fk))
│    %10  = Core.apply_type(Main.Optimizers.Array, Main.Optimizers.Float64, 1)
│    %11  = Core.apply_type(Main.Optimizers.Array, Main.Optimizers.Float64, 1)
│    %12  = Core.apply_type(Main.Optimizers.Array, Main.Optimizers.Float64, 1)
│    %13  = Core.apply_type(Main.Optimizers.Tuple, %10, %11, %12, Main.Optimizers.Float64, Main.Optimizers.Float64, Main.Optimizers.Int64)
│    %14  = Main.Optimizers.size(A)
│    %15  = Base.indexed_iterate(%14, 1)
│           m = Core.getfield(%15, 1)
│           @_11 = Core.getfield(%15, 2)
│    %18  = Base.indexed_iterate(%14, 2, @_11)
│           n = Core.getfield(%18, 1)
│    %20  = Main.Optimizers.size(c, 1)
│    %21  = %20 ==